<a href="https://colab.research.google.com/github/jialun1221/scRNA-seq/blob/main/Preprocessing1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing and clustering PD astrocytes
### Part 1. Data selection

In May 2017, this started out as a demonstration that Scanpy would allow to reproduce most of Seurat's [guided clustering tutorial](http://satijalab.org/seurat/pbmc3k_tutorial.html) ([Satija et al., 2015](https://doi.org/10.1038/nbt.3192)).

We gratefully acknowledge Seurat's authors for the tutorial! In the meanwhile, we have added and removed a few pieces.

The data consist of *3k PBMCs from a Healthy Donor* and are freely available from 10x Genomics ([here](http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz) from this [webpage](https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/pbmc3k)). On a unix system, you can uncomment and run the following to download and unpack the data. The last line creates a directory for writing processed data.

In this notebook, we will compute ***Data Selection***. We will drop the cells that cotain Lewy Body Dementia, and create a new AnnData object that contains only PD and control cells. All other features of the original AnnData will reamin.

In [ ]:
#import packages 
!pip install scanpy
import numpy as np
import pandas as pd
import scanpy as sc

In [ ]:
#make directories for file storage
!mkdir data
!mkdir write

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘write’: File exists


In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.21.6 scipy==1.7.3 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.12.2 python-igraph==0.9.11 pynndescent==0.5.7


In [ ]:
# results_file = 'write/pd_astro.h5ad'  # the file that will store the analysis results

In [ ]:
# pca_file = 'write/pca.h5ad'

In [ ]:
# write anndata with PCA stored in adata.obsm
# pca_in_obsm = 'write/pca_in_obsm.h5ad'

In [ ]:
#file to store new Anndata object 
new_anndata = 'write/new_anndata.h5ad'

Read in the count matrix into an [AnnData](https://anndata.readthedocs.io/en/latest/anndata.AnnData.html) object, which holds many slots for annotations and different representations of the data. It also comes with its own HDF5-based file format: `.h5ad`.

In [ ]:
!pip install --quiet scvi-tools[tutorials]
import scvi

INFO:pytorch_lightning.utilities.seed:Global seed set to 0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
adata = scvi.data.read_h5ad("drive/MyDrive/PD_astro.h5ad") 

In [ ]:
adata

AnnData object with n_obs × n_vars = 33506 × 41625
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Cell_Subtype', 'Cell_Type', 'disease__ontology_label', 'organ__ontology_label'
    var: 'features'



**Note**
    
Start with some basic checking.

In [ ]:
adata.var_names #this gives genes!

Index(['RP11-34P13.3', 'FAM138A', 'OR4F5', 'RP11-34P13.7', 'RP11-34P13.8',
       'RP11-34P13.14', 'RP11-34P13.9', 'FO538757.3', 'FO538757.2',
       'AP006222.2',
       ...
       'RNU2-71P', 'RNA5SP252', 'AC116533.2', 'AC114494.1', 'RN7SL424P',
       'RN7SL739P', 'MIR4502', 'RNU4-10P', 'RN7SL865P', 'RNU6-191P'],
      dtype='object', length=41625)

In [ ]:
adata.obs_names #this are labels

Index(['pPDsHSrSNxi3482d200429PosB_CTCTCGATCACACCCT-1',
       'pPDsHSrSNxi3482d200429PosB_GATCCCTTCTGGACCG-1',
       'pPDsHSrSNxi3482d200429PosB_ATCGTAGTCGGTCGAC-1',
       'pPDsHSrSNxi3482d200429PosB_GGGACAAAGTGCAACG-1',
       'pPDsHSrSNxi3482d200429PosB_TTGTTCATCGCAATTG-1',
       'pPDsHSrSNxi3482d200429PosB_ATGGTTGCAGCGTACC-1',
       'pPDsHSrSNxi3482d200429PosB_ACGGAAGGTGAGTAAT-1',
       'pPDsHSrSNxi3482d200429PosB_TCCGAAAGTTGCTGAT-1',
       'pPDsHSrSNxi3482d200429PosB_GCAGGCTAGGTCGAGT-1',
       'pPDsHSrSNxi3482d200429PosB_CTGTGGGTCATCGACA-1',
       ...
       'pPDsHSrSNxi1963d200429PosE_TTTGACTCAATCGTCA-1',
       'pPDsHSrSNxi1963d200429PosE_CATTGAGTCCTTCTAA-1',
       'pPDsHSrSNxi1963d200429PosE_CAACAACCACCTGCGA-1',
       'pPDsHSrSNxi1963d200429PosE_ATTCTTGGTTCCACGG-1',
       'pPDsHSrSNxi1963d200429PosE_CATAAGCAGATACAGT-1',
       'pPDsHSrSNxi1963d200429PosE_AACAAGAAGGGTCACA-1',
       'pPDsHSrSNxi3298d200429PosB_TGGGCTGAGATACCAA-1',
       'pPDsHSrSNxi3298d200429PosB_CA

In [ ]:
#Check how many rows are unwanted data.
adata.obs.loc[adata.obs['disease__ontology_label'].str.contains("Lewy body dementia", case=False)]

,orig.ident,nCount_RNA,nFeature_RNA,Cell_Subtype,Cell_Type,disease__ontology_label,organ__ontology_label
pPDsHSrSNxi4775d200429PosA_CTCAACCCAGCGAACA-1,86,18376.0,6077,Astro_GBP2_SPOCD1,astro,Lewy body dementia,substantia nigra pars compacta
pPDsHSrSNxi4775d200429PosA_AGATCCACAGAGTGAC-1,86,10638.0,4211,Astro_GJB6_OXTR,astro,Lewy body dementia,substantia nigra pars compacta
pPDsHSrSNxi4775d200429PosA_TGTAACGTCCTACCAC-1,86,10009.0,4048,Astro_GJB6_OXTR,astro,Lewy body dementia,substantia nigra pars compacta
pPDsHSrSNxi4775d200429PosA_CTCATGCCAACGTAAA-1,86,9735.0,3777,Astro_GJB6_OXTR,astro,Lewy body dementia,substantia nigra pars compacta
pPDsHSrSNxi4775d200429PosA_CTCAGAAAGGGAACAA-1,86,8617.0,3824,Astro_GJB6_OXTR,astro,Lewy body dementia,substantia nigra pars compacta
...,...,...,...,...,...,...,...
pPDsHSrSNxi2569d200429DAPIB_GACTCTCCACATAGCT-1,30,4764.0,2632,Ependyma_ZBBX,astro,Lewy body dementia,substantia nigra pars compacta
pPDsHSrSNxi2569d200429DAPIB_CTGCAGGCATGACAAA-1,30,2836.0,1762,Ependyma_ZBBX,astro,Lewy body dementia,substantia nigra pars compacta
pPDsHSrSNxi2569d200429DAPIB_AGACTCAGTCTCCTGT-1,30,2632.0,1515,Ependyma_ZBBX,astro,Lewy body dementia,substantia nigra pars compacta
pPDsHSrSNxi2569d200429DAPIB_TCATATCTCCATAGGT-1,30,2380.0,1478,Ependyma_ZBBX,astro,Lewy body dementia,substantia nigra pars compacta


###Data selection

In [ ]:
!pip install matplotlib==3.1.3
from numpy import inf

Drop the Lewy body dementia:

In [ ]:
adata.obs = adata.obs.reset_index() #Set index for the labels
k = adata.obs #create a variable for further uses (a DataFrame)
# print(k)

In [ ]:
y = k.index[k['disease__ontology_label'] == 'Lewy body dementia'].tolist() #get the index that contains the Lewy Body Dementia samples, stored in variable y (a list)
# print(y)

In [ ]:
m = adata.X.toarray() #convert sparse matrix X to array

Conduct data selection separately in adata.X and adata.obs. 

In [ ]:
m = np.delete(m, obj = y, axis=0) #delete rows that contain Lewy Body Dementia according to the previously generated index stored in y

In [ ]:
#drop command for adata.obs
adata.obs.drop(adata.obs.index[adata.obs['disease__ontology_label'] == 'Lewy body dementia'], inplace=True)
adata.obs

,index,orig.ident,nCount_RNA,nFeature_RNA,Cell_Subtype,Cell_Type,disease__ontology_label,organ__ontology_label
0,pPDsHSrSNxi3482d200429PosB_CTCTCGATCACACCCT-1,59,29990.0,7831,Astro_VIM_TNFSRF12A,astro,normal,substantia nigra pars compacta
1,pPDsHSrSNxi3482d200429PosB_GATCCCTTCTGGACCG-1,59,28671.0,7291,Astro_VIM_TNFSRF12A,astro,normal,substantia nigra pars compacta
2,pPDsHSrSNxi3482d200429PosB_ATCGTAGTCGGTCGAC-1,59,26473.0,7010,Astro_VIM_TNFSRF12A,astro,normal,substantia nigra pars compacta
3,pPDsHSrSNxi3482d200429PosB_GGGACAAAGTGCAACG-1,59,22933.0,6906,Astro_VIM_TNFSRF12A,astro,normal,substantia nigra pars compacta
4,pPDsHSrSNxi3482d200429PosB_TTGTTCATCGCAATTG-1,59,22217.0,6975,Astro_VIM_TNFSRF12A,astro,normal,substantia nigra pars compacta
...,...,...,...,...,...,...,...,...
33501,pPDsHSrSNxi1963d200429PosE_AACAAGAAGGGTCACA-1,15,6054.0,2974,Ependyma_ZBBX,astro,Parkinson disease,substantia nigra pars compacta
33502,pPDsHSrSNxi3298d200429PosB_TGGGCTGAGATACCAA-1,36,6898.0,2719,Astro_GLYATL2,astro,normal,substantia nigra pars compacta
33503,pPDsHSrSNxi3298d200429PosB_CAGCCAGCATCTAACG-1,36,2766.0,1381,Astro_GUCY1A2,astro,normal,substantia nigra pars compacta
33504,pPDsHSrSNxi3298d200429PosB_GCCTGTTGTATTCTCT-1,36,1798.0,992,Astro_GJB6_OXTR,astro,normal,substantia nigra pars compacta


##making new AnnData object ##

In [ ]:
pip install anndata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Command for making a new AnnData object. For each parameter, need to make a deep copy of the original object.
new = sc.AnnData(X = m,
  obs = adata.obs.copy(),
  var = adata.var.copy(),
  uns = adata.uns.copy(),
  obsm = adata.obsm.copy(),
  varm = adata.varm.copy(),
  layers = adata.layers.copy(),
  raw = adata.raw.copy(),
  dtype = "float32",
  shape = None,
  #filename = NULL,
  #filemode = NULL,
  obsp = adata.obsp.copy(),
  varp = adata.varp
  )
#varp = adata.varp.copy() would give me error but direct assignment would not

/usr/local/lib/python3.7/dist-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [ ]:
#A random line that I found necessary for the object to work. 
new.__dict__['_raw'].__dict__['_var'] = adata.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})

In [ ]:
new.write(new_anndata)

In [ ]:
print(adata.X.shape, new.X.shape) #Now the new AnnData object is generated. Check the dimension!

(33506, 41625) (26535, 41625)


A new AnnData object is created, and stored in the Colab disk. Navigate to the folder button on the left side panel, and click on "write", you will find the `new_anndata.h5ad file` here. Please either download it to your local disk, then upload to your google drive; or move to your drive folder by dragging it to the `drive` folder. 

---
The purpose of creating a new AnnData is to keep the accessibility of other features, stored in `adata.obsm`, `adata.varm`, etc. 


**Basic filtering:**

In [ ]:
sc.pp.filter_cells(new, min_genes=200) #Minimum number of genes expressed required for a cell to pass filtering.
sc.pp.filter_genes(new, min_cells=3) #Minimum number of cells expressed required for a gene to pass filtering.

Let's assemble some information about mitochondrial genes, which are important for quality control.

Citing from "Simple Single Cell" workflows [(Lun, McCarthy & Marioni, 2017)](https://master.bioconductor.org/packages/release/workflows/html/simpleSingleCell.html#examining-gene-level-metrics):

> High proportions are indicative of poor-quality cells (Islam et al. 2014; Ilicic et al. 2016), possibly because of loss of cytoplasmic RNA from perforated cells. The reasoning is that mitochondria are larger than individual transcript molecules and less likely to escape through tears in the cell membrane.

With `pp.calculate_qc_metrics`, we can compute many metrics very efficiently.

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

A violin plot of some of the computed quality measures:

* the number of genes expressed in the count matrix
* the total counts per cell
* the percentage of counts in mitochondrial genes

In [ ]:
import scanpy as sc
import seaborn as sns


adata.var["mito"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mito"], inplace=True)
sns.jointplot(
    data=adata.obs,
    x="log1p_total_counts",
    y="log1p_n_genes_by_counts",
    kind="hex",
)

In [ ]:
sns.histplot(adata.obs["pct_counts_mito"])

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
adata.obs.n_genes_by_counts

In [ ]:
adata.obs.total_counts

Remove cells that have too many mitochondrial genes expressed or too many total counts:

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

Actually do the filtering by slicing the `AnnData` object.

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]

Total-count normalize (library-size correct) the data matrix $\mathbf{X}$ to 10,000 reads per cell, so that counts become comparable among cells.

In [ ]:
adata

In [ ]:
adata.obs['disease__ontology_label']

**Drop the Lewy Body Dementia before normalization**

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

In [ ]:
adata.obs.total_counts #this is the total_counts after normalized

Logarithmize the data:

In [ ]:
sc.pp.log1p(adata)

Identify highly-variable genes.

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(adata)

Set the `.raw` attribute of the AnnData object to the normalized and logarithmized raw gene expression for later use in differential testing and visualizations of gene expression. This simply freezes the state of the AnnData object.

<div class="alert alert-info">

**Note**

You can get back an `AnnData` of the object in `.raw` by calling `.raw.to_adata()`.

</div>    

In [ ]:
adata.raw = adata

In [ ]:
adata.raw

<div class="alert alert-info">

**Note**
    
If you don't proceed below with correcting the data with `sc.pp.regress_out` and scaling it via `sc.pp.scale`, you can also get away without using `.raw` at all.

The result of the previous highly-variable-genes detection is stored as an annotation in `.var.highly_variable` and auto-detected by PCA and hence, `sc.pp.neighbors` and subsequent manifold/graph tools. In that case, the step *actually do the filtering* below is unnecessary, too.

</div>

Actually do the filtering

In [ ]:
adata = adata[:, adata.var.highly_variable] #3799 highly variable genes

In [ ]:
print(adata)

Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. Scale the data to unit variance.

In [ ]:
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

Scale each gene to unit variance. Clip values exceeding standard deviation 10. 

In [ ]:
sc.pp.scale(adata, max_value=10)

## Principal component analysis

Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.

In [ ]:
pcadata = sc.tl.pca(adata, n_comps = 50, svd_solver='arpack')

In [ ]:
type(pcadata)

In [ ]:
pd.DataFrame(adata.obsm['X_pca'].components)

In [ ]:
print(pcadata)
print(type(pcadata))
pcadata.shape

**Concatenate** the pcadata with labels

In [ ]:
print(adata.X)
print(type(adata))
adata.shape

In [ ]:
#Replace the adata.X with pca data
adata.X = anndata.obsm['raw'].copy()

In [ ]:
print(adata)

In [ ]:
adata.write(pca_in_obsm)
#NoW i got the new anndata structure with data after PCA stored in adata.obsm

In [ ]:
pcadata = adata.obsm['X_pca']
pcadata = pd.DataFrame(adata.obsm['X_pca'])

In [ ]:
pcadata = adata.obsm['X_pca']
pcadata #the PCA data is just an array of X

In [ ]:
adata.varm['PCs']

In [ ]:
pcadata = sc.AnnData(pcadata)

In [ ]:
type(pcadata)

In [ ]:
pcadata.write(pca_file)

In [ ]:
type(pcadata)

In [ ]:
type(adata)

In [ ]:
adata.var.highly_variable

In [ ]:
print(adata.raw)

In [ ]:
adata.obs

In [ ]:
type(pcadata)

We can make a scatter plot in the PCA coordinates, but we will not use that later on.

In [ ]:
sc.pl.pca(adata, color='CST3')

Let us inspect the contribution of single PCs to the total variance in the data. This gives us information about how many PCs we should consider in order to compute the neighborhood relations of cells, e.g. used in the clustering function  `sc.tl.louvain()` or tSNE `sc.tl.tsne()`. In our experience, often a rough estimate of the number of PCs does fine.

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

Save the result.

In [ ]:
adata.write(results_file)

In [ ]:
adata

## Computing the neighborhood graph

Let us compute the neighborhood graph of cells using the PCA representation of the data matrix. You might simply use default values here. For the sake of reproducing Seurat's results, let's take the following values.

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

## Embedding the neighborhood graph

We suggest embedding the graph in two dimensions using UMAP ([McInnes et al., 2018](https://arxiv.org/abs/1802.03426)), see below. It is potentially more faithful to the global connectivity of the manifold than tSNE, i.e., it better preserves trajectories. In some ocassions, you might still observe disconnected clusters and similar connectivity violations. They can usually be remedied by running:

```
tl.paga(adata)
pl.paga(adata, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
tl.umap(adata, init_pos='paga')
```

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['CST3', 'NKG7', 'PPBP'])

As we set the `.raw` attribute of `adata`, the previous plots showed the "raw" (normalized, logarithmized, but uncorrected) gene expression. You can also plot the scaled and corrected gene expression by explicitly stating that you don't want to use `.raw`.

In [ ]:
sc.pl.umap(adata, color=['CST3', 'NKG7', 'PPBP'], use_raw=False)

## Clustering the neighborhood graph

As with Seurat and many other frameworks, we recommend the Leiden graph-clustering method (community detection based on optimizing modularity) by [Traag *et al.* (2018)](https://scanpy.readthedocs.io/en/latest/references.html#traag18). Note that Leiden clustering directly clusters the neighborhood graph of cells, which we already computed in the previous section.

In [ ]:
sc.tl.leiden(adata)

Plot the clusters, which agree quite well with the result of Seurat.

In [ ]:
sc.pl.umap(adata, color=['leiden', 'CST3', 'NKG7'])

Save the result.

In [ ]:
adata.write(results_file)

## Finding marker genes

Let us compute a ranking for the highly differential genes in each cluster. For this, by default, the `.raw` attribute of AnnData is used in case it has been initialized before. The simplest and fastest method to do so is the t-test.

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
sc.settings.verbosity = 2  # reduce the verbosity

The result of a [Wilcoxon rank-sum (Mann-Whitney-U)](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test) test is very similar. We recommend using the latter in publications, see e.g., [Sonison & Robinson (2018)](https://doi.org/10.1038/nmeth.4612). You might also consider much more powerful differential testing packages like MAST, limma, DESeq2 and, for python, the recent diffxpy.

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

Save the result.

In [ ]:
adata.write(results_file)

As an alternative, let us rank genes using logistic regression. For instance, this has been suggested by [Natranos et al. (2018)](https://doi.org/10.1101/258566). The essential difference is that here, we use a multi-variate appraoch whereas conventional differential tests are uni-variate. [Clark et al. (2014)](https://doi.org/10.1186/1471-2105-15-79) has more details.

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='logreg')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

With the exceptions of *IL7R*, which is only found by the t-test and *FCER1A*, which is only found by the other two appraoches, all marker genes are recovered in all approaches.

Louvain Group | Markers | Cell Type
---|---|---
0 | IL7R | CD4 T cells
1 | CD14, LYZ | CD14+ Monocytes
2 | MS4A1 |	B cells
3 | CD8A |	CD8 T cells
4 | GNLY, NKG7 | 	NK cells
5 | FCGR3A, MS4A7 |	FCGR3A+ Monocytes
6 | FCER1A, CST3 |	Dendritic Cells
7 | PPBP |	Megakaryocytes

Let us also define a list of marker genes for later reference.

In [ ]:
marker_genes = ['IL7R', 'CD79A', 'MS4A1', 'CD8A', 'CD8B', 'LYZ', 'CD14',
                'LGALS3', 'S100A8', 'GNLY', 'NKG7', 'KLRB1',  
                'FCGR3A', 'MS4A7', 'FCER1A', 'CST3', 'PPBP']

Reload the object that has been save with the Wilcoxon Rank-Sum test result.

In [ ]:
adata = sc.read(results_file)

Show the 10 top ranked genes per cluster 0, 1, ..., 7 in a dataframe.

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(5)

Get a table with the scores and groups.

In [ ]:
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(5)

Compare to a single cluster: 

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', groups=['0'], reference='1', method='wilcoxon')
sc.pl.rank_genes_groups(adata, groups=['0'], n_genes=20)

If we want a more detailed view for a certain group, use `sc.pl.rank_genes_groups_violin`.

In [ ]:
sc.pl.rank_genes_groups_violin(adata, groups='0', n_genes=8)

Reload the object with the computed differential expression (i.e. DE via a comparison with the rest of the groups):

In [ ]:
adata = sc.read(results_file)

In [ ]:
sc.pl.rank_genes_groups_violin(adata, groups='0', n_genes=8)

If you want to compare a certain gene across groups, use the following.

In [ ]:
sc.pl.violin(adata, ['CST3', 'NKG7', 'PPBP'], groupby='leiden')

Actually mark the cell types.

In [ ]:
new_cluster_names = [
    'CD4 T', 'CD14 Monocytes',
    'B', 'CD8 T', 
    'NK', 'FCGR3A Monocytes',
    'Dendritic', 'Megakaryocytes']
adata.rename_categories('leiden', new_cluster_names)

In [ ]:
sc.pl.umap(adata, color='leiden', legend_loc='on data', title='', frameon=False, save='.pdf')

Now that we annotated the cell types, let us visualize the marker genes.

In [ ]:
sc.pl.dotplot(adata, marker_genes, groupby='leiden');

There is also a very compact violin plot.

In [ ]:
sc.pl.stacked_violin(adata, marker_genes, groupby='leiden', rotation=90);

During the course of this analysis, the AnnData accumlated the following annotations.

In [ ]:
adata

In [ ]:
adata.write(results_file, compression='gzip')  # `compression='gzip'` saves disk space, but slows down writing and subsequent reading

Get a rough overview of the file using `h5ls`, which has many options - for more details see [here](https://github.com/theislab/scanpy_usage/blob/master/170505_seurat/info_h5ad.md). The file format might still be subject to further optimization in the future. All reading functions will remain backwards-compatible, though.

If you want to share this file with people who merely want to use it for visualization, a simple way to reduce the file size is by removing the dense scaled and corrected data matrix. The file still contains the raw data used in the visualizations in `adata.raw`.

In [ ]:
adata.raw.to_adata().write('./write/pbmc3k_withoutX.h5ad')

If you want to export to "csv", you have the following options:

In [ ]:
# Export single fields of the annotation of observations
# adata.obs[['n_counts', 'louvain_groups']].to_csv(
#     './write/pbmc3k_corrected_louvain_groups.csv')

# Export single columns of the multidimensional annotation
# adata.obsm.to_df()[['X_pca1', 'X_pca2']].to_csv(
#     './write/pbmc3k_corrected_X_pca.csv')

# Or export everything except the data using `.write_csvs`.
# Set `skip_data=False` if you also want to export the data.
# adata.write_csvs(results_file[:-5], )